In [407]:
import pandas as pd
from skimage import color
from skimage.io import imread
import numpy as np

In [408]:
df = pd.read_csv("./local_path.csv")

In [409]:
for i in range(len(df)):
    s = df.local_path.loc[i]
    if "DS_Store" in s:
        df = df.drop(i)
        
df.reset_index(drop=True,inplace=True)

In [410]:
len(df)

7148

In [411]:
brightness = []
n_channels = []

for file in df.local_path:    
    img = imread(file)
    channels = len(img.shape)
    n_channels.append(channels)
    img_gray = color.rgb2gray(img)
    
    # because the bw are [0,255] and the color are [0,1]
    if channels == 2:
        val_mean = np.mean(img_gray) / 255    
    else:
        val_mean = np.mean(img_gray)
        
    brightness.append(val_mean)
    
df['val'] = brightness
df['n_channels'] = n_channels

In [412]:
i_strings = []
for i in range(len(df)):
    s = df.local_path.loc[i]
    i_string = s[45:].split("/")[0]
    i_strings.append(i_string)

In [413]:
df['i_strings'] = i_strings

In [414]:
from PIL import Image, ImageDraw

In [415]:
import math

In [416]:
side = int(math.ceil( np.sqrt(len(df)) ))

In [417]:
thumb_side = 64

In [418]:
px_w = side * thumb_side
px_h = side * thumb_side

In [419]:
df.sort("val",ascending=False,inplace=True)

In [420]:
df.reset_index(drop=True,inplace=True)

In [421]:
canvas = Image.new('RGB',(px_w, px_h),(50,50,50))
x,y = range(side) * side, np.repeat(range(side),side)
df['x'] = x[:len(df)]
df['y'] = y[:len(df)]

In [422]:
for i in range(len(df)):
    im = Image.open(df.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    x_coord = df['x'].loc[i] * thumb_side
    y_coord = df['y'].loc[i] * thumb_side
    
    pos_i = (40,5)
    pos_i_string = (5,50)
    i_string = df.i_strings.loc[i]
    
    draw = ImageDraw.Draw(im)
    
    if i < 5600:
        draw.text(pos_i, str(i),fill="black")
        draw.text(pos_i_string,i_string,fill="black")
    else:
        draw.text(pos_i, str(i),fill="white")
        draw.text(pos_i_string,i_string,fill="white")
    
    canvas.paste(im,(x_coord,y_coord))
    
canvas.save("./dupes_text_clean.png")

In [423]:
# dangerous move here, but deleting doubles...

In [424]:
import os

In [425]:
"""
for i in range(len(df)):
    if i==0:
        val = df.val.loc[i]
        i_string = df.i_strings.loc[i]
    else:
        tmp = df.val.loc[i]
        temp = df.i_strings.loc[i]
        if tmp==val and temp==i_string:
            os.remove(df.local_path.loc[i])
        else:
            val = df.val.loc[i]
            i_string = df.i_strings.loc[i]
"""

'\nfor i in range(len(df)):\n    if i==0:\n        val = df.val.loc[i]\n        i_string = df.i_strings.loc[i]\n    else:\n        tmp = df.val.loc[i]\n        temp = df.i_strings.loc[i]\n        if tmp==val and temp==i_string:\n            os.remove(df.local_path.loc[i])\n        else:\n            val = df.val.loc[i]\n            i_string = df.i_strings.loc[i]\n'